In [2]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

b = chat.predict("How many planets are there?")

b

/Users/ben/Documents/GitHub/Nomadcoders/Program/GPT/fullstack-gpt/env/lib/python3.10/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/ben/Documents/GitHub/Nomadcoders/Program/GPT/fullstack-gpt/env/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


('As of current scientific knowledge, there are 8 officially recognized planets in our solar system. These are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')